# Wrestle the Android App Store Data into Beautiful Looking Charts with Plotly

Have you ever thought about building your own an iOS or Android app? If so, then you probably have wondered about how things work in the app stores. Today we'll replicate some of the app store analytics provided by companies like App Annie or Sensor Tower that helps inform development and app marketing strategies for many companies. This stuff is BIG business!

We will compare thousands of apps in the Google Play Store so that we can gain insight into:

- How competitive different app categories (e.g., Games, Lifestyle, Weather) are

- Which app category offers compelling opportunities based on its popularity

- How many downloads you would give up by making your app paid vs. free

- How much you can reasonably charge for a paid app

- Which paid apps have had the highest revenue

- How many paid apps will recoup their development costs based on their sales revenue



In [90]:
import pandas as pd

df_apps = pd.read_csv('apps.csv')

# Data Cleaning: Removin NaN Values and Duplicates

The first step as always is getting a better idea about what we're dealing with.

### Preliminary Data Exploration

How many rows and columns does df_apps have? What are the column names? What does the data look like? Look at a random sample of 5 different rows with [.sample()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html)

In [91]:
# How many rows and columns does df_apps have?
df_apps.shape

# 10841 rows, 12 columns

(10841, 12)

We are working with a fairly large DataFrame this time.

10841 rows and 12 columns.

In [92]:
df_apps.head()

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres,Last_Updated,Android_Ver
0,Ak Parti Yardım Toplama,SOCIAL,NaN,0,8.7,0,Paid,$13.99,Teen,Social,"July 28, 2017",4.1 and up
1,Ain Arabic Kids Alif Ba ta,FAMILY,NaN,0,33.0,0,Paid,$2.99,Everyone,Education,"April 15, 2016",3.0 and up
2,Popsicle Launcher for Android P 9.0 launcher,PERSONALIZATION,NaN,0,5.5,0,Paid,$1.49,Everyone,Personalization,"July 11, 2018",4.2 and up
3,Command & Conquer: Rivals,FAMILY,NaN,0,19.0,0,NaN,0,Everyone 10+,Strategy,"June 28, 2018",Varies with device
4,CX Network,BUSINESS,NaN,0,10.0,0,Free,0,Everyone,Business,"August 6, 2018",4.1 and up


We can already see that there are some data issues that we need to fix. In the Ratings and Type columns there are NaN (Not a number values) and in the Price column we have dollar signs that will cause problems.



In [93]:
# What are the column names? 
print('Columns: ')
for col in df_apps.columns:
    print(f' - {col}')

Columns: 
 - App
 - Category
 - Rating
 - Reviews
 - Size_MBs
 - Installs
 - Type
 - Price
 - Content_Rating
 - Genres
 - Last_Updated
 - Android_Ver


In [94]:
# What does the data look like? Look at a random sample of 5 different rows with .sample()
df_apps.sample(5)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres,Last_Updated,Android_Ver
9195,Metal Soldiers 2,GAME,4.4,153381,36.00,"10,000,000",Free,0,Teen,Action,"June 18, 2018",4.1 and up
472,All-night drugstore CI & Price,HEALTH_AND_FITNESS,NaN,0,5.10,10,Paid,$0.99,Everyone,Health & Fitness,"November 27, 2017",1.6 and up
2487,CG Sanyukt Bharti Pariksha 2018,FAMILY,4.3,14,4.30,"1,000",Free,0,Teen,Education,"January 10, 2018",2.3 and up
4357,Popeyes® App,FOOD_AND_DRINK,3.7,61,11.00,"50,000",Free,0,Everyone,Food & Drink,"July 23, 2018",4.1 and up
5042,Money Manager Ex for Android,FINANCE,4.2,2695,9.15,"100,000",Free,0,Everyone,Finance,"February 26, 2018",Varies with device


Remove the columns called Last_Updated and Android_Version from the DataFrame. We will not use these columns.



In [95]:
df_apps = df_apps.drop('Last_Updated', axis=1) #axis = 0 for rows, axis = 1 for columns
df_apps = df_apps.drop('Android_Ver', axis=1)

In [96]:
print('Columns: ')
for col in df_apps.columns:
    print(f' - {col}')

Columns: 
 - App
 - Category
 - Rating
 - Reviews
 - Size_MBs
 - Installs
 - Type
 - Price
 - Content_Rating
 - Genres


How many rows have a NaN value (not-a-number) in the Rating column? Create DataFrame called df_apps_clean that does not include these rows.

In [97]:
df_apps.Rating.isna().sum()

np.int64(1474)

In [98]:
# df_apps_clean = df_apps[df_apps.Rating.notna()]
df_apps_clean = df_apps.dropna()

In [99]:
df_apps_clean.shape

(9367, 10)

This leaves us with 9,367 entries in our DataFrame. But there may be other problems with the data too:

Are there any duplicates in data? Check for duplicates using the [.duplicated()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html) function. How many entries can you find for the "Instagram" app? Use [.drop_duplicates()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html) to remove any duplicates from `df_apps_clean`.



In [100]:
df_apps_clean[df_apps_clean.duplicated()]

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
946,420 BZ Budeze Delivery,MEDICAL,5.0,2,11.0,100,Free,0,Mature 17+,Medical
1133,MouseMingle,DATING,2.7,3,3.9,100,Free,0,Mature 17+,Dating
1196,"Cardiac diagnosis (heart rate, arrhythmia)",MEDICAL,4.4,8,6.5,100,Paid,$12.99,Everyone,Medical
1231,Sway Medical,MEDICAL,5.0,3,22.0,100,Free,0,Everyone,Medical
1247,Chat Kids - Chat Room For Kids,DATING,4.7,6,4.9,100,Free,0,Mature 17+,Dating
...,...,...,...,...,...,...,...,...,...,...
10802,Skype - free IM & video calls,COMMUNICATION,4.1,10484169,3.5,"1,000,000,000",Free,0,Everyone,Communication
10809,Instagram,SOCIAL,4.5,66577313,5.3,"1,000,000,000",Free,0,Teen,Social
10826,Google Drive,PRODUCTIVITY,4.4,2731211,4.0,"1,000,000,000",Free,0,Everyone,Productivity
10832,Google News,NEWS_AND_MAGAZINES,3.9,877635,13.0,"1,000,000,000",Free,0,Teen,News & Magazines


In [101]:
df_apps_clean[df_apps_clean.App == 'Instagram']

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
10806,Instagram,SOCIAL,4.5,66577313,5.3,"1,000,000,000",Free,0,Teen,Social
10808,Instagram,SOCIAL,4.5,66577446,5.3,"1,000,000,000",Free,0,Teen,Social
10809,Instagram,SOCIAL,4.5,66577313,5.3,"1,000,000,000",Free,0,Teen,Social
10810,Instagram,SOCIAL,4.5,66509917,5.3,"1,000,000,000",Free,0,Teen,Social


In [102]:
df_apps_clean[df_apps_clean.App == 'Instagram'].duplicated()

10806    False
10808    False
10809     True
10810    False
dtype: bool

In [103]:
df_apps_clean = df_apps_clean.drop_duplicates()

In [104]:
df_apps_clean[df_apps_clean.App == 'Instagram']

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
10806,Instagram,SOCIAL,4.5,66577313,5.3,"1,000,000,000",Free,0,Teen,Social
10808,Instagram,SOCIAL,4.5,66577446,5.3,"1,000,000,000",Free,0,Teen,Social
10810,Instagram,SOCIAL,4.5,66509917,5.3,"1,000,000,000",Free,0,Teen,Social


We need to provide the column names that should be used in the comparison to identify duplicates. 

In [105]:
df_apps_clean = df_apps_clean.drop_duplicates(subset=['App', 'Type', 'Price'])

In [106]:
df_apps_clean[df_apps_clean.App == 'Instagram']

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
10806,Instagram,SOCIAL,4.5,66577313,5.3,"1,000,000,000",Free,0,Teen,Social


In [107]:
df_apps_clean.shape

(8199, 10)

This leaves us with 8,199 entries after removing duplicates. 

### What else should I know about the data?

So we can see that 13 different features were originally scraped from the Google Play Store.

- Obviously, the data is just a sample out of all the Android apps. It doesn't include all Android apps of which there are millions.

- I’ll assume that the sample is representative of the App Store as a whole. This is not necessarily the case as, during the web scraping process, this sample was served up based on geographical location and user behaviour of the person who scraped it - in our case Lavanya Gupta.

- The data was compiled around 2017/2018. The pricing data reflect the price in USD Dollars at the time of scraping. (developers can offer promotions and change their app’s pricing).

- I’ve converted the app’s size to a floating-point number in MBs. If data was missing, it has been replaced by the average size for that category.

- The installs are not the exact number of installs. If an app has 245,239 installs then Google will simply report an order of magnitude like 100,000+. I’ve removed the '+' and we’ll assume the exact number of installs in that column for simplicity.



# Preliminary Exploration: The Highest Ratings, Most Reviews, and Largest Size

- Identify which apps are the highest rated. What problem might you encounter if you rely exclusively on ratings alone to determine the quality of an app?

In [108]:
df_apps_clean.sort_values(by='Rating', ascending=False)[:11]

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
21,KBA-EZ Health Guide,MEDICAL,5.0,4,25.0,1,Free,0,Everyone,Medical
1230,Sway Medical,MEDICAL,5.0,3,22.0,100,Free,0,Everyone,Medical
1227,AJ Men's Grooming,LIFESTYLE,5.0,2,22.0,100,Free,0,Everyone,Lifestyle
1224,FK Dedinje BGD,SPORTS,5.0,36,2.6,100,Free,0,Everyone,Sports
1223,CB VIDEO VISION,PHOTOGRAPHY,5.0,13,2.6,100,Free,0,Everyone,Photography
1222,"Beacon Baptist Jupiter, FL",LIFESTYLE,5.0,14,2.6,100,Free,0,Everyone,Lifestyle
1214,BV Mobile Apps,PRODUCTIVITY,5.0,3,4.8,100,Free,0,Everyone,Productivity
2680,Florida Wildflowers,FAMILY,5.0,5,69.0,"1,000",Free,0,Everyone,Education
1206,ADS-B Driver,TOOLS,5.0,2,6.3,100,Paid,$1.99,Everyone,Tools
2750,"Superheroes, Marvel, DC, Comics, TV, Movies News",COMICS,5.0,34,12.0,"5,000",Free,0,Everyone,Comics


Only apps with very few reviews (and a low number on installs) have perfect 5 star ratings (most likely by friends and family).

- What's the size in megabytes (MB) of the largest Android apps in the Google Play Store. Based on the data, do you think there could be a limit in place or can developers make apps as large as they please?

In [109]:
df_apps_clean.sort_values(by='Size_MBs', ascending=False).head()

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
9942,Talking Babsy Baby: Baby Games,LIFESTYLE,4.0,140995,100.0,"10,000,000",Free,0,Everyone,Lifestyle;Pretend Play
10687,Hungry Shark Evolution,GAME,4.5,6074334,100.0,"100,000,000",Free,0,Teen,Arcade
9943,Miami crime simulator,GAME,4.0,254518,100.0,"10,000,000",Free,0,Mature 17+,Action
9944,Gangster Town: Vice District,FAMILY,4.3,65146,100.0,"10,000,000",Free,0,Mature 17+,Simulation
3144,Vi Trainer,HEALTH_AND_FITNESS,3.6,124,100.0,"5,000",Free,0,Everyone,Health & Fitness


Here we can clearly see that there seems to be an upper bound of 100 MB for the size of an app. A quick google search would also have revealed that this limit is imposed by the Google Play Store itself. It’s interesting to see that a number of apps actually hit that limit exactly.


- Which apps have the highest number of reviews? Are there any paid apps among the top 50?

In [110]:
df_apps_clean.sort_values(by='Reviews', ascending=False).head(n=50)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
10805,Facebook,SOCIAL,4.1,78158306,5.30,"1,000,000,000",Free,0,Teen,Social
10785,WhatsApp Messenger,COMMUNICATION,4.4,69119316,3.50,"1,000,000,000",Free,0,Everyone,Communication
10806,Instagram,SOCIAL,4.5,66577313,5.30,"1,000,000,000",Free,0,Teen,Social
10784,Messenger – Text and Video Chat for Free,COMMUNICATION,4.0,56642847,3.50,"1,000,000,000",Free,0,Everyone,Communication
10650,Clash of Clans,GAME,4.6,44891723,98.00,"100,000,000",Free,0,Everyone 10+,Strategy
10744,Clean Master- Space Cleaner & Antivirus,TOOLS,4.7,42916526,3.40,"500,000,000",Free,0,Everyone,Tools
10835,Subway Surfers,GAME,4.5,27722264,76.00,"1,000,000,000",Free,0,Everyone 10+,Arcade
10828,YouTube,VIDEO_PLAYERS,4.3,25655305,4.65,"1,000,000,000",Free,0,Teen,Video Players & Editors
10746,"Security Master - Antivirus, VPN, AppLock, Boo...",TOOLS,4.7,24900999,3.40,"500,000,000",Free,0,Everyone,Tools
10584,Clash Royale,GAME,4.6,23133508,97.00,"100,000,000",Free,0,Everyone 10+,Strategy


If you look at the number of reviews, you can find the most popular apps on the Android App Store. These include the usual suspects: Facebook, WhatsApp, Instagram etc. What’s also notable is that the list of the top 50 most reviewed apps does not include a single paid app! 🤔



# Data Visualisation with Plotly: Create Pie and Donut Charts

All Android apps have a content rating like “Everyone” or “Teen” or “Mature 17+”. Let’s take a look at the distribution of the content ratings in our dataset and see how to visualise it with [plotly](https://plotly.com/python/) - a popular data visualisation library that you can use alongside or instead of Matplotlib.

First, we’ll count the number of occurrences of each rating with `.value_counts()`



In [111]:
ratings = df_apps_clean.Content_Rating.value_counts()
ratings

Content_Rating
Everyone           6621
Teen                912
Mature 17+          357
Everyone 10+        305
Adults only 18+       3
Unrated               1
Name: count, dtype: int64

The first step in creating charts with plotly is to import plotly.express. This is the fastest way to create a beautiful graphic with a minimal amount of code in plotly.

## Import Statements


In [112]:
import plotly.express as px

To create a pie chart we simply call `px.pie()` and then `.show()` the resulting figure. Plotly refers to all their figures, be they line charts, bar charts, or pie charts as graph_objects.



In [113]:
fig = px.pie(labels=ratings.index, values=ratings.values)
fig.show()

Let’s customise our pie chart. Looking at the [.pie() documentation](https://plotly.com/python-api-reference/generated/plotly.express.pie.html) we see a number of parameters that we can set, like title or names.

If you’d like to configure other aspects of the chart, that you can’t see in the list of parameters, you can call a method called `.update_traces()`. In plotly lingo, “traces” refer to graphical marks on a figure. Think of “traces” as collections of attributes. Here we update the traces to change how the text is displayed.



In [114]:
fig = px.pie(
    labels=ratings.index, 
    values=ratings.values,
    title="Content Rating",
    names=ratings.index
)

fig.update_traces(textposition='outside', textinfo='percent+label')

fig.show()

To create a donut 🍩 chart, we can simply add a value for the hole argument:

In [115]:
fig = px.pie(
    labels=ratings.index,
    values=ratings.values,
    title="Content Rating",
    names=ratings.index,
    hole=0.6,
)

fig.update_traces(
    textposition='inside',
    textfont_size=15,
    textinfo='percent',
)

fig.show()

# Numeric Type Conversions for the Installations & Price Data

How many apps had over 1 billion (that's right - BILLION) installations? How many apps just had a single install?

1. Check the datatype of the Installs column.

2. Count the number of apps at each level of installations.

3. Convert the number of installations (the Installs column) to a numeric data type. Hint: this is a 2-step process. You'll have to make sure you remove non-numeric characters first.

In [116]:
df_apps_clean.Installs.describe()

count          8199
unique           19
top       1,000,000
freq           1417
Name: Installs, dtype: object

In [117]:
type(df_apps_clean["Installs"].loc[21])

str

In [118]:
df_apps_clean[["App", "Installs"]].groupby("Installs").count()

,App
Installs,
1,3
"1,000",698
"1,000,000",1417
"1,000,000,000",20
10,69
"10,000",988
"10,000,000",933
100,303
"100,000",1096


In [119]:
df_apps_clean.Installs = df_apps_clean.Installs.str.replace(",", "")

In [120]:
df_apps_clean.Installs.max()

'500000000'

In [121]:
df_apps_clean.Installs = pd.to_numeric(df_apps_clean.Installs)

In [122]:
df_apps_clean[df_apps_clean.Installs >= 1000000000].count()

App               20
Category          20
Rating            20
Reviews           20
Size_MBs          20
Installs          20
Type              20
Price             20
Content_Rating    20
Genres            20
dtype: int64

In [123]:
df_apps_clean[df_apps_clean.Installs == 1].count()

App               3
Category          3
Rating            3
Reviews           3
Size_MBs          3
Installs          3
Type              3
Price             3
Content_Rating    3
Genres            3
dtype: int64

Let's examine the Price column more closely.

Convert the price column to numeric data. Then investigate the top 20 most expensive apps in the dataset.



Remove all apps that cost more than $250 from the `df_apps_clean` DataFrame.



Add a column called 'Revenue_Estimate' to the DataFrame. This column should hold the price of the app times the number of installs. What are the top 10 highest-grossing paid apps according to this estimate? Out of the top 10, how many are games?



In [125]:
df_apps_clean.Price.describe()

count     8199
unique      73
top          0
freq      7595
Name: Price, dtype: object

In [128]:
print(df_apps_clean.Price)

21           0
28       $1.49
47       $0.99
82           0
99           0
         ...  
10824        0
10828        0
10829        0
10831        0
10835        0
Name: Price, Length: 8199, dtype: object


In [129]:
df_apps_clean.Price = df_apps_clean.Price.str.replace("$", "")
df_apps_clean.Price = pd.to_numeric(df_apps_clean.Price)

In [133]:
df_apps_clean.sort_values("Price", ascending=False).head(20)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
3946,I'm Rich - Trump Edition,LIFESTYLE,3.6,275,7.300000,10000,Paid,400.00,Everyone,Lifestyle
2461,I AM RICH PRO PLUS,FINANCE,4.0,36,41.000000,1000,Paid,399.99,Everyone,Finance
4606,I Am Rich Premium,FINANCE,4.1,1867,4.700000,50000,Paid,399.99,Everyone,Finance
3145,I am rich(premium),FINANCE,3.5,472,0.942383,5000,Paid,399.99,Everyone,Finance
3554,💎 I'm rich,LIFESTYLE,3.8,718,26.000000,10000,Paid,399.99,Everyone,Lifestyle
5765,I am rich,LIFESTYLE,3.8,3547,1.800000,100000,Paid,399.99,Everyone,Lifestyle
1946,I am rich (Most expensive app),FINANCE,4.1,129,2.700000,1000,Paid,399.99,Teen,Finance
2775,I Am Rich Pro,FAMILY,4.4,201,2.700000,5000,Paid,399.99,Everyone,Entertainment
3221,I am Rich Plus,FAMILY,4.0,856,8.700000,10000,Paid,399.99,Everyone,Entertainment
3114,I am Rich,FINANCE,4.3,180,3.800000,5000,Paid,399.99,Everyone,Finance


In [134]:
df_apps_clean = df_apps_clean[df_apps_clean.Price <= 250]

In [136]:
df_apps_clean['Revenue_Estimate'] = df_apps_clean.Installs * df_apps_clean.Price

In [138]:
df_apps_clean.sort_values("Revenue_Estimate", ascending=False).head(10)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres,Revenue_Estimate
9220,Minecraft,FAMILY,4.5,2376564,19.000000,10000000,Paid,6.99,Everyone 10+,Arcade;Action & Adventure,69900000.0
8825,Hitman Sniper,GAME,4.6,408292,29.000000,10000000,Paid,0.99,Mature 17+,Action,9900000.0
7151,Grand Theft Auto: San Andreas,GAME,4.4,348962,26.000000,1000000,Paid,6.99,Mature 17+,Action,6990000.0
7477,Facetune - For Free,PHOTOGRAPHY,4.4,49553,48.000000,1000000,Paid,5.99,Everyone,Photography,5990000.0
7977,Sleep as Android Unlock,LIFESTYLE,4.5,23966,0.851562,1000000,Paid,5.99,Everyone,Lifestyle,5990000.0
6594,DraStic DS Emulator,GAME,4.6,87766,12.000000,1000000,Paid,4.99,Everyone,Action,4990000.0
6082,Weather Live,WEATHER,4.5,76593,4.750000,500000,Paid,5.99,Everyone,Weather,2995000.0
7954,Bloons TD 5,FAMILY,4.6,190086,94.000000,1000000,Paid,2.99,Everyone,Strategy,2990000.0
7633,Five Nights at Freddy's,GAME,4.6,100805,50.000000,1000000,Paid,2.99,Teen,Action,2990000.0
6746,Card Wars - Adventure Time,FAMILY,4.3,129603,23.000000,1000000,Paid,2.99,Everyone 10+,Card;Action & Adventure,2990000.0


7 are games but the category can is FAMILY or GAME

The top spot of the highest-grossing paid app goes to … Minecraft at close to $70 million. It’s quite interesting that Minecraft (along with Bloons and Card Wars) is actually listed in the Family category rather than in the Game category. If we include these titles, we see that 7 out the top 10 highest-grossing apps are games. The Google Play Store seems to be quite flexible with its category labels.
